In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
d1 = pd.read_csv("/content/drive/MyDrive/PD_scans/neurocon/neurocon_patients.tsv",sep = '\t')
d2 = pd.read_csv("/content/drive/MyDrive/PD_scans/taowu/taowu_patients.tsv",sep = '\t')

In [ ]:
d1 = d1.status
d2 = d2.disease_duration 

In [ ]:
d1 = d1.to_numpy()

In [ ]:
d2 = d2.to_numpy()

In [ ]:
d2[d2 > 0] = 1

In [ ]:
y_1 = []
y_2 = []

In [ ]:
for i in d1:
  if i=='NORMAL':
    y_1.append(0)
  else:
    y_1.append(1)

for i in d2:
  if i==1.0:
    y_2.append(1)
  else:
    y_2.append(0)

In [ ]:
y = y_1+ y_2

In [ ]:
import glob

In [ ]:
files_neurocon = glob.glob('/content/drive/MyDrive/PD_scans/neurocon/**/anat/*.nii',recursive = True)

In [ ]:
files_neurocon_gz = glob.glob('/content/drive/MyDrive/PD_scans/neurocon/**/anat/*.gz',recursive = True)

In [ ]:
files_neurocon = files_neurocon + files_neurocon_gz

In [ ]:
sort_files_neurocon = files_neurocon

In [ ]:
sort_files_neurocon.sort(key = lambda x: x.split('/')[6])

In [ ]:
files_taowu = glob.glob('/content/drive/MyDrive/PD_scans/taowu/**/anat/*.gz',recursive = True)

In [ ]:
sort_files_taowu = files_taowu

In [ ]:
sort_files_taowu.sort(key = lambda x: x.split('/')[6])

In [ ]:
import matplotlib.pyplot as plt
import itertools

In [ ]:
import nibabel as nib

In [ ]:
def resize_data(data):
    initial_size_x = data.shape[0]
    initial_size_y = data.shape[1]
    initial_size_z = data.shape[2]

    new_size_x = 144
    new_size_y = 144
    new_size_z = 96

    delta_x = initial_size_x / new_size_x
    delta_y = initial_size_y / new_size_y
    delta_z = initial_size_z / new_size_z

    new_data = np.zeros((new_size_x, new_size_y, new_size_z))

    for x, y, z in itertools.product(range(new_size_x),
                                     range(new_size_y),
                                     range(new_size_z)):
        new_data[x][y][z] = data[int(x * delta_x)][int(y * delta_y)][int(z * delta_z)]

    return new_data

In [ ]:
X_neurocon =[]

In [ ]:
for i in sort_files_neurocon:
  X_neurocon.append(resize_data((nib.load(i).get_fdata())))

In [ ]:
X_taowu =[]

In [ ]:
for i in sort_files_taowu:
  X_taowu.append(resize_data((nib.load(i).get_fdata())))

In [ ]:
len(X_neurocon), len(X_taowu)

(43, 40)

In [ ]:
X = X_neurocon + X_taowu

In [ ]:
import random
from scipy import ndimage
import tensorflow as tf

In [ ]:
def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        
        angle = random.choice(angles)
        
        volume = ndimage.rotate(volume, angle, reshape=False)
        
        return volume

In [ ]:
#for i in X_neurocon:
#  X_neurocon.append(scipy_rotate(i))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
with strategy.scope():
  inputs = keras.Input((144, 144, 96, 1))

  x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
  x = layers.MaxPool3D(pool_size=2)(x)
  x = layers.BatchNormalization()(x)

  x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
  x = layers.MaxPool3D(pool_size=2)(x)
  x = layers.BatchNormalization()(x)

  x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
  x = layers.MaxPool3D(pool_size=2)(x)
  x = layers.BatchNormalization()(x)

  x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
  x = layers.MaxPool3D(pool_size=2)(x)
  x = layers.BatchNormalization()(x)

  x = layers.GlobalAveragePooling3D()(x)
  x = layers.Dense(units=512, activation="relu")(x)
  x = layers.Dropout(0.3)(x)

  outputs = layers.Dense(units=1, activation="sigmoid")(x)

In [ ]:
with strategy.scope():
  model = keras.Model(inputs, outputs, name="3dcnn")

In [ ]:
model.summary()

In [ ]:
len(X), len(y)

In [ ]:
X = np.asarray(X)
y = np.asarray(y)

In [ ]:
with strategy.scope():
  initial_learning_rate = 0.0001
  lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [ ]:
with strategy.scope():
  model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op NoOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Floor in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pow in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

In [ ]:
epochs = 100
model.fit(
    X,y,
    validation_split=0.2,
    epochs=epochs,
    shuffle=True,
    batch_size=4, 
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

Streaming output truncated to the last 5000 lines.
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4285 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4285 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /j